In [36]:
import pandas as pd
import random
import numpy as np
import glob
import os
import pickle
F1 = open('E:\\SZ\\2019\\1203\\Snapshot.pkl', 'rb')
SZ = pickle.load(F1)
SZ['time'] = (SZ['OrigTime'] - 20191203000000000).astype(int)
date = pd.DataFrame(pd.date_range(start='2019-06-10 08:30:00', end='2019-06-10 18:00:00', freq='s'), columns=["Orig"])
date["time"] = date["Orig"].apply(lambda x: int(x.strftime("%H%M%S"))*1000)
date["group"] = date["time"]//30000
SZ["group"] = SZ["time"]//30000
gl = date[((date["time"] >= 93000000) & (date["time"] <= 113000000))|((date["time"] >= 130000000) & (date["time"] <= 145700000))]["group"].unique()
print(set(gl) - set(SZ["group"].unique()))
print(date[date["group"].isin(list(set(gl) - set(SZ["group"].unique())))])

set()
Empty DataFrame
Columns: [Orig, time, group]
Index: []


In [1]:
import pandas as pd
import random
import numpy as np
import glob
import os
import pickle
F1 = open('E:\\SZ\\2019\\0610\\Snapshot.pkl', 'rb')
k1 = pickle.load(F1)
k1["time"] = (k1["OrigTime"] - 20191203000000000)
k1 = k1[(k1["StockID"] < 4000) | (k1["StockID"] > 300000)]
data2 = k1[(k1["time"] > 92500000) & (k1["time"] <= 145700000) & (k1["cum_volume"] > 0)]
del k1
data2["volume_update"] = data2.groupby("StockID")["cum_volume"].apply(lambda x: x - x.shift(1))
data2["time1"] = pd.to_datetime(data2["OrigTime"].apply(lambda x: str(x)[:-3]))
data2["time_interval"] = data2.groupby("StockID")["time1"].apply(lambda x: x - x.shift(1))
data2["time_interval"] = data2["time_interval"].apply(lambda x: x.seconds)
data2 = pd.merge(data2, data2[data2["volume_update"] > 0].groupby("StockID")["time_interval"].describe()["75%"].reset_index(), on="StockID")
data2 = data2.rename(columns={"75%": "75%_time"})
data2 = pd.merge(data2, data2[data2["volume_update"] > 0].groupby("StockID")["volume_update"].describe().reset_index(), on="StockID")
data2["75%"] = data2["75%"].apply(lambda x: max(1000, x))
data2["indicator"] = 0
data2.loc[data2[data2["time"] >= 130000000].groupby("StockID").head(1).index, 'indicator'] = 1
data2.loc[data2[data2["time"] >= 93000000].groupby("StockID").head(1).index, 'indicator'] = 1
data2 = data2[data2["indicator"] != 1]
data2[(data2["time_interval"] > 180) & (data2["volume_update"] >= data2["75%"]) & (data2["75%_time"] <= 4)]

,Date,OrigTime,SendTime_x,ercvtime_x,dbtime_x,ChannelNo,StockID,SecurityIDsource,MDStreamID,ask1p,...,75%_time,count,mean,std,min,25%,50%,75%,max,indicator
3456,20190610,20190610135824000,20190610135824301,20190610140610577,20190610140610578,1011,1,102,10,12.30,...,3.0,4460.0,25310.045516,73257.642169,12.0,3300.0,9300.0,21800.0,1568100.0,0
8083,20190610,20190610135806000,20190610135806261,20190610140551438,20190610140551438,1014,2,102,10,27.81,...,3.0,4388.0,11912.906563,27358.782893,100.0,1300.0,4000.0,11700.0,595100.0,0
17842,20190610,20190610135824000,20190610135824301,20190610140610579,20190610140610579,1011,7,102,10,6.68,...,3.0,1444.0,3506.160665,6308.432096,16.0,400.0,1300.0,3900.0,77500.0,0
21523,20190610,20190610135824000,20190610135824301,20190610140610580,20190610140610581,1011,8,102,10,3.73,...,3.0,1806.0,6721.280731,22029.724699,100.0,600.0,2000.0,6200.0,715158.0,0
25247,20190610,20190610135806000,20190610135806297,20190610140551528,20190610140551529,1012,9,102,10,5.51,...,3.0,2045.0,7346.536919,64524.917433,9.0,500.0,1700.0,5000.0,2828405.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6564352,20190610,20190610135824000,20190610135824851,20190610140612806,20190610140612807,1013,300772,102,10,16.31,...,3.0,3228.0,3709.987299,7660.655952,64.0,600.0,1700.0,4000.0,233201.0,0
6568653,20190610,20190610135809000,20190610135809820,20190610140555095,20190610140555095,1014,300773,102,10,51.65,...,3.0,2527.0,1312.897507,2388.357070,25.0,200.0,500.0,1400.0,48500.0,0
6573151,20190610,20190610135806000,20190610135806823,20190610140552551,20190610140552551,1014,300775,102,10,27.64,...,3.0,4408.0,7547.432169,33359.803532,36.0,1200.0,2600.0,6408.0,1569154.0,0
6577196,20190610,20190610135806000,20190610135806845,20190610140552662,20190610140552663,1012,300776,102,10,118.30,...,3.0,1861.0,619.399785,1095.713921,88.0,100.0,300.0,1000.0,23309.0,0
